<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install category_encoders==2.0.0

In [0]:
# Import libraries
import pandas as pd
import numpy as np
import math

from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.metrics import mean_absolute_error

import statsmodels.api as sm

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from geopy.distance import vincenty as get_geodesic_distance

In [44]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/SF-SFR-Sales-Final2d.csv')

# Rename subdistr_desc to neighborhood
df = df.rename(columns={'subdist_no': 'nid', 'subdist_desc': 'neighborhood'})

# Check the data
print(df.shape)
df.head(5)

(23711, 39)


,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,on_market_date,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,3/14/13,4,1095000,3/22/13,1260000,8,3.5,4,T,Per Tax Records,2691,NaN,0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN"
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,12/3/15,144,1250000,4/25/16,1075000,9,3.0,4,T,Per Tax Records,2437,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,5/18/17,36,1395000,8/17/17,1525000,9,5.0,5,D,Per Architect,2597,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/27/10,42,725000,8/20/10,715000,5,1.0,2,T,Per Tax Records,1312,NaN,0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK"
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,6/7/12,24,1595000,7/13/12,1595000,8,4.0,4,T,Per Tax Records,3307,NaN,0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK"


In [45]:
# Baseline of sale_price
df['sale_price'].mean()

1371454.081312471

In [46]:
# Train, test split on date of 01/01/2018
df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True)
df['year_sold'] = df['sale_date'].dt.year
df['month_sold'] = df['sale_date'].dt.month
df['day_sold'] = df['sale_date'].dt.day

df['on_market_date'] = pd.to_datetime(df['on_market_date'], infer_datetime_format=True)
df['year_on_market'] = df['on_market_date'].dt.year
df['month_on_market'] = df['on_market_date'].dt.month
df['day_on_market'] = df['on_market_date'].dt.day

low_cutoff = 2008
high_cutoff = 2018
train = df[(df['year_sold'] >= low_cutoff) & (df['year_sold'] < high_cutoff)]
test  = df[df['year_sold'] >= high_cutoff]
print(train.shape)
print(test.shape)

(21487, 45)
(2224, 45)


In [47]:
# Wrangle the data for train and test
def engineer_features(X):
  
  # Impute mean for null long/lat/elev based on mean of of neighborhood
  def feature_calc(feature, nid, f_dict):
    if math.isnan(feature):
      if (nid) in f_dict:
        new_feature = f_dict[nid]
        return new_feature
    else:
      return feature
    
    return feature
  
  temp = X[~X['longitude'].isna()].groupby(['nid'])['longitude'].mean()
  long_dict = dict(temp)
  X['longitude'] = X.apply(lambda x: feature_calc(x['longitude'], x['nid'], long_dict), axis=1)
  
  temp = X[~X['latitude'].isna()].groupby(['nid'])['latitude'].mean()
  lat_dict = dict(temp)
  X['latitude'] = X.apply(lambda x: feature_calc(x['latitude'], x['nid'], lat_dict), axis=1)
  
  temp = X[~X['elevation'].isna()].groupby(['nid'])['elevation'].mean()
  elev_dict = dict(temp)
  X['elevation'] = X.apply(lambda x: feature_calc(x['elevation'], x['nid'], elev_dict), axis=1)
  
  # Engineer a cluster feature based on long/lat clusters
  kmeans = KMeans(n_clusters=10, n_jobs=-1)
  X['cluster'] = kmeans.fit_predict(X[['longitude', 'latitude']])
  X['cluster'] = kmeans.predict(X[['longitude', 'latitude']])
  
  X['zip'] = X['zip'].astype(int)
  
  # Fill rooms zero values by adding beds and baths
  def room_calc(rooms_val, beds_val, baths_val):
    if rooms_val == 0:
      total = beds_val + baths_val
    else:
      total = rooms_val
      
    return total
  
  X['rooms'] = X.apply(lambda x: room_calc(x['rooms'], x['beds'], x['baths']), axis=1)
  
  # Fill baths zero values by adding beds and baths
  X.loc[(X['baths'] == 0)] = 1
  
  # Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
  averagesf_data = X.groupby('beds').sf.mean()

  # Use average sf by bedroom for each 0 value in each bedroom group up to 9 bedrooms
  for i in range(0, 9): 
    X.loc[(X['sf'] == 0) & (X['beds'] == i), 'sf'] = averagesf_data.loc[i]

  # Use 10,000sf for anything over 9 bedrooms
  X.loc[X['sf'] == 0, 'sf'] = 10000
  X = X.astype({'sf': int})
  
  # Fill lot_sf zero values by using lot_acres to calc
  def lotsf_calc(lotsf_val, lotacres_val):
    if lotsf_val == 0:
      total = lotacres_val * 43560
    else:
      total = lotsf_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  
  # Fill lot_acres zero values by using lot_sf to calc
  def lotacres_calc(lotacres_val, lotsf_val):
    if lotacres_val == 0:
      total = lotsf_val / 43560
    else:
      total = lotacres_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  X['lot_acres'] = X.apply(lambda x: lotacres_calc(x['lot_acres'], x['lot_sf']), axis=1)
  
  # Features with nan's that should be zero's
  cols_with_nans = ['lot_sf', 'lot_acres', 'year_built', 'zoning']
  for col in cols_with_nans:
    X[col] = X[col].replace(np.nan, 0)
    
  # Features with numeric zero's that should be nan's
  #cols_with_czeros = ['lot_desc', 'drive_side', 'parking', 'park_leased',
  #                    'shopping', 'transportation', 'type', 'views']
  #for col in cols_with_czeros:
  #  X[col] = X[col].replace('0', np.nan)
    
  # Engineer a price_sf column
  X['price_sf'] = X['sale_price'] / X['sf']
  
  # Drop unneeded columns
  unneeded_columns = ['sale_date', 'on_market_date', 'full_address', 'city', 'state', 'street_no', 'street_name',
                      'street_suffix', 'district_desc', 'neighborhood', 'sf_source', 'sf_source_decs']
  X = X.drop(columns=unneeded_columns)
  
  return X

train = engineer_features(train)
test = engineer_features(test)

train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view

,longitude,latitude,elevation,zip,area,district_no,nid,cdom,orig_list_price,sale_price,rooms,baths,beds,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,month_sold,day_sold,year_on_market,month_on_market,day_on_market,cluster,price_sf
0,-122.50965,37.780280,200.83,94121,1050,1,1050,4,1095000,1260000,8.0,3.5,4,2691,0.0000,0.0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,3,22,2013,3,14,5,468.227425
1,-122.50929,37.762608,23.21,94122,2030,2,2030,144,1250000,1075000,9.0,3.0,4,2437,0.0689,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2016,4,25,2015,12,3,0,441.116126
2,-122.50929,37.762608,23.21,94122,2030,2,2030,36,1395000,1525000,9.0,5.0,5,2597,0.0689,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2017,8,17,2017,5,18,0,587.216018
3,-122.50924,37.777330,189.11,94121,1050,1,1050,42,725000,715000,5.0,1.0,2,1312,0.0000,0.0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010,8,20,2010,6,27,5,544.969512
4,-122.50919,37.776695,175.89,94121,1050,1,1050,24,1595000,1595000,8.0,4.0,4,3307,0.0000,0.0,1951,0,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012,7,13,2012,6,7,5,482.310251


In [48]:
# Train
cutoff = 2017
temp=train.copy()
train = temp[temp['year_sold'] < 2017]
val  = temp[temp['year_sold'] >= 2017]
print(train.shape, val.shape, test.shape)

(19227, 35) (2260, 35) (2224, 35)


In [49]:
# Encode and fit a linear regression model

target = 'sale_price'

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
      ce.OrdinalEncoder(),
      LinearRegression()
  )

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
  
mae = mean_absolute_error(y_val, y_pred)

print(f'Test MAE: ${mae:.0f}')

print ('Validation Accuracy', pipeline.score(X_val, y_val))

print ('Test Accuracy', pipeline.score(X_test, y_test))

X_train.head()

Test MAE: $213804
Validation Accuracy 0.894559699715372
Test Accuracy 0.8109367765024088


,longitude,latitude,elevation,zip,area,district_no,nid,cdom,orig_list_price,rooms,baths,beds,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,month_sold,day_sold,year_on_market,month_on_market,day_on_market,cluster,price_sf
0,-122.509650,37.780280,200.83,94121,1050,1,1050,4,1095000,8.0,3.5,4,2691,0.0000,0.0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,3,22,2013,3,14,5,468.227425
1,-122.509290,37.762608,23.21,94122,2030,2,2030,144,1250000,9.0,3.0,4,2437,0.0689,3000.0,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0,2016,4,25,2015,12,3,0,441.116126
3,-122.509240,37.777330,189.11,94121,1050,1,1050,42,725000,5.0,1.0,2,1312,0.0000,0.0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010,8,20,2010,6,27,5,544.969512
4,-122.509190,37.776695,175.89,94121,1050,1,1050,24,1595000,8.0,4.0,4,3307,0.0000,0.0,1951,0,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012,7,13,2012,6,7,5,482.310251
6,-122.509186,37.761005,23.69,94122,2030,2,2030,76,600000,4.0,1.0,3,1168,0.0000,0.0,1947,0,"RGLR,LEVL",PVDW,"ATCH,GARG,ATDR",0,2,0,1BLK,ATAC,0,2009,5,19,2009,2,18,0,453.767123


In [0]:
pipeline.fit(X_test, y_test)
y_pred = pipeline.predict(X_test)

In [51]:
final = test.copy()
final['prediction'] = y_pred
final['difference'] = final['sale_price'] - final['prediction']
final.head()

,longitude,latitude,elevation,zip,area,district_no,nid,cdom,orig_list_price,sale_price,rooms,baths,beds,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,month_sold,day_sold,year_on_market,month_on_market,day_on_market,cluster,price_sf,prediction,difference
5,-122.509190,37.776695,175.89,94121,1050,1,1050,13,2749000,3310000,8.0,4.0,4,3735,0.0909,3959.0,1951,0,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,6,21,2018,5,17,3,886.211513,3.703283e+06,-393283.346923
7,-122.509090,37.759580,23.45,94122,2030,2,2030,131,1595000,1304000,5.0,2.0,3,1520,0.0606,2639.0,1900,RM1,"RGLR,LEVL",PVDW,UNCV,ONST,1,1BLK,1BLK,"SMAT,3STR","PNRM,PRTL,PARK,GRDN,HILL",2018,11,28,2018,7,6,8,857.894737,1.316820e+06,-12819.849687
9,-122.509056,37.758010,23.01,94122,2030,2,2030,15,1298000,1830000,7.0,3.0,4,2120,0.0705,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,5,23,2018,4,30,8,863.207547,1.853186e+06,-23185.586309
15,-122.509040,37.774418,47.10,94121,1050,1,1050,20,1295000,1650000,6.0,2.0,3,1520,0.0689,3000.0,1957,RH2,0,0,"ATCH,ATDR,INAC",ONST,1,0,0,0,0,2018,3,23,2018,2,15,3,1085.526316,1.506513e+06,143486.793937
17,-122.509020,37.758636,22.70,94122,2030,2,2030,19,895000,1300000,3.0,1.0,2,1087,0.0286,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,7,18,2018,6,14,8,1195.952162,1.571346e+06,-271345.745718


In [54]:
final.head(25)

,longitude,latitude,elevation,zip,area,district_no,nid,cdom,orig_list_price,sale_price,rooms,baths,beds,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,month_sold,day_sold,year_on_market,month_on_market,day_on_market,cluster,price_sf,prediction,difference
5,-122.509190,37.776695,175.89,94121,1050,1,1050,13,2749000,3310000,8.0,4.0,4,3735,0.0909,3959.0,1951,0,0,0,"ATCH,GARG,ATDR,INAC",ONST,3,3BLK,1BLK,2STR,"PNRM,CTYL,WATR,SFRN,OCEN,PARK,GRDN,TWNP",2018,6,21,2018,5,17,3,886.211513,3.703283e+06,-3.932833e+05
7,-122.509090,37.759580,23.45,94122,2030,2,2030,131,1595000,1304000,5.0,2.0,3,1520,0.0606,2639.0,1900,RM1,"RGLR,LEVL",PVDW,UNCV,ONST,1,1BLK,1BLK,"SMAT,3STR","PNRM,PRTL,PARK,GRDN,HILL",2018,11,28,2018,7,6,8,857.894737,1.316820e+06,-1.281985e+04
9,-122.509056,37.758010,23.01,94122,2030,2,2030,15,1298000,1830000,7.0,3.0,4,2120,0.0705,3072.0,1948,RM1,"RGLR,OCNF",0,GARG,ONST,1,2BLK,2BLK,FULL,GRDN,2018,5,23,2018,4,30,8,863.207547,1.853186e+06,-2.318559e+04
15,-122.509040,37.774418,47.10,94121,1050,1,1050,20,1295000,1650000,6.0,2.0,3,1520,0.0689,3000.0,1957,RH2,0,0,"ATCH,ATDR,INAC",ONST,1,0,0,0,0,2018,3,23,2018,2,15,3,1085.526316,1.506513e+06,1.434868e+05
17,-122.509020,37.758636,22.70,94122,2030,2,2030,19,895000,1300000,3.0,1.0,2,1087,0.0286,1245.0,1944,RM1,0,"PVDW,PVSW",ATCH,ONST,1,1BLK,1BLK,0,0,2018,7,18,2018,6,14,8,1195.952162,1.571346e+06,-2.713457e+05
23,-122.508980,37.773680,40.08,94121,1050,1,1050,3,1100000,1300000,6.0,1.5,2,1326,0.0619,2696.0,1942,RH2,"RGLR,LEVL,FNCD","PVDW,PVSW",GARG,ONST,2,1BLK,1BLK,"JR,2STR",0,2018,4,9,2018,3,5,3,980.392157,1.091285e+06,2.087153e+05
46,-122.508370,37.752396,25.53,94122,2030,2,2030,13,1495000,2129000,7.0,3.0,3,1520,0.0557,2426.0,1944,RH2,0,0,"GARG,INAC",ONST,2,0,0,0,"WATR,OCEN,GRDN",2018,3,14,2018,2,8,8,1400.657895,2.449628e+06,-3.206276e+05
57,-122.508270,37.778465,225.66,94121,1050,1,1050,5,1588000,1865000,6.0,2.0,3,1800,0.0689,3000.0,1931,RH1,RGLR,"PVDW,PVSW","ATCH,GARG,ATDR,INAC",ONST,2,0,1BLK,"FULL,2STR",PRTL,2018,5,22,2018,4,26,3,1036.111111,1.762329e+06,1.026709e+05
62,-122.508250,37.777084,193.52,94121,1050,1,1050,10,2995000,3200000,9.5,4.5,5,3221,0.0619,2696.0,1941,RH1,"RGLR,LEVL,FNCD","PVDW,PVSW","ATCH,GARG,ATDR,INAC",ONST,1,0,0,2STR,"CTYL,OCEN,MNTN,TWNP",2018,3,13,2018,2,17,3,993.480286,3.339610e+06,-1.396100e+05
63,-122.508240,37.751440,24.55,94122,2030,2,2030,0,1800000,1900000,5.0,1.0,2,995,0.0449,1957.0,1944,RH2,0,0,0,ONST,2,0,0,0,0,2018,3,30,2018,3,30,8,1909.547739,3.067519e+06,-1.167519e+06


In [59]:
def neighbor_mean(beds, source_longitude, source_latitude):

  print(beds, source_longitude, source_latitude)
  
  source_lonlat = source_longitude, source_latitude
  source_table = train[train['beds'] == beds]
  
  print(source_table.head())

  target_table = pd.DataFrame(source_table, columns = ['longitude', 'latitude', 'sale_price']) 
  
  print(target_table.head())

  def get_distance(row):
      target_lonlat = row['longitude'], row['latitude']
      return get_geodesic_distance(target_lonlat, source_lonlat).meters

  target_table['distance'] = target_table.apply(get_distance, axis=1)
  
  print(target_table.head())

  # Get the nearest 4 locations
  nearest_target_table = target_table.sort_values(['distance'])[:3]

  nearest_target_table

  return nearest_target_table['price'].mean()

final['mean_neighbor_price'] = final.apply(lambda x: neighbor_mean(x['beds'], x['longitude'], x['latitude']), axis=1)
final.head()

4 -122.50918999999999 37.776695000000004
    longitude   latitude  elevation  ...  day_on_market  cluster    price_sf
0  -122.50965  37.780280     200.83  ...             14        5  468.227425
1  -122.50929  37.762608      23.21  ...              3        0  441.116126
4  -122.50919  37.776695     175.89  ...              7        5  482.310251
28 -122.50879  37.755703      24.39  ...             18        0  660.480349
39 -122.50856  37.762170      26.53  ...              4        0  402.336145

[5 rows x 35 columns]
    longitude   latitude  sale_price
0  -122.50965  37.780280     1260000
1  -122.50929  37.762608     1075000
4  -122.50919  37.776695     1595000
28 -122.50879  37.755703     1210000
39 -122.50856  37.762170      620000
4 -122.50918999999999 37.776695000000004
    longitude   latitude  elevation  ...  day_on_market  cluster    price_sf
0  -122.50965  37.780280     200.83  ...             14        5  468.227425
1  -122.50929  37.762608      23.21  ...              3  

/usr/local/lib/python3.6/dist-packages/geopy/point.py:81: UserWarning:

Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.



ValueError: ignored